In [1]:

#-----------------------------------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")
import time as t
from datetime import datetime, timedelta, date 
from dateutil.relativedelta import *
from netCDF4 import Dataset   
import glob
import requests 
import pandas as pd
import xarray as xr
import requests 
import functools
import operator
import wget
import os
import gc
import numpy as np
import pygrib
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy import signal
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import adfuller
import datetime
import holidays
import xarray as xr


import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
import shapefile
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import cartopy, cartopy.crs as ccrs   
import regionmask
from shapely.geometry import Point, Polygon

from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker 
import scipy.ndimage as ndimage
from PIL import Image
from glob import glob

import matplotlib.colors as colors
import matplotlib 



#-----METPY---------------------
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.interpolate import cross_section
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
file_med_merge = '/mnt/c/scripts/temas/stream_plot/dado_climato/merge_all_files.grb2'



file = xr.open_dataset(file_med_merge)

#Correção da coordenada time 
#time = start do modelo 
#valid time = tempo de analise e previsão  (time,step)


file = file.stack(valid_time_=['time', 'step']).swap_dims({'valid_time_': 'valid_time'}).drop_vars(['time']).drop(['valid_time_','step'])

lat_slice = slice(20,-90)
file = file.sel(latitude=lat_slice)

file.coords['longitude'] = (file.coords['longitude'] + 180) % 360 - 180 
file = file.sortby(file.longitude)

#Climatologia 
med = file.groupby('valid_time.month').mean()




#save grb2 
med.to_netcdf('./climatology_montlhy_mean_psi200.gbr2')